# Data Analysis 

In [8]:
import os

print(os.listdir("/Users/yasmineakaichi/fed-popper/fedpopper"))

['noisy-alzheimer_acetyl_part1', 'client1.py', 'server.py', 'trains2', 'trains_part2', 'carcinogenesis', 'trains_part3', 'synthesis-contains', 'trains2_part3', 'iggp-rps_part3', 'iggp-rps_part2', 'trains2_part2', 'fedpopper_results_client2.csv', 'fedpopper_results_client3.csv', 'fedpopper_results_client1.csv', 'test.py', 'zendo1_part1', 'reverse', 'noisy-alzheimer_acetyl', 'noisy-alzheimer_acetyl_part2', 'noisy-alzheimer_acetyl_part3', 'trains_part1', 'iggp-rps', 'client3.py', 'iggp-rps_part1', 'trains2_part1', 'zendo1_part3', 'analysis.ipynb', 'trains', 'zendo1_part2', 'zendo1', 'client2.py']


In [12]:
import os
import re
import pandas as pd

ROOT = "/Users/yasmineakaichi/fed-popper/fedpopper"

def count_pos_neg(filepath):
    with open(filepath, "r") as f:
        txt = f.read()
    pos = len(re.findall(r"\bpos\(", txt))
    neg = len(re.findall(r"\bneg\(", txt))
    return pos, neg

rows = []

for root, dirs, files in os.walk(ROOT):
    if "exs.pl" in files:
        ex_file = os.path.join(root, "exs.pl")
        pos, neg = count_pos_neg(ex_file)

        # dataset = le dossier parent du dossier contenant exs.pl
        parts = root.split(os.sep)
        dataset = parts[-2] if len(parts) >= 2 else parts[-1]
        partition = parts[-1]

        rows.append({
            "dataset": dataset,
            "partition": partition,
            "pos": pos,
            "neg": neg,
            "total": pos + neg,
            "path": ex_file
        })

df = pd.DataFrame(rows)
df


,dataset,partition,pos,neg,total,path
0,fedpopper,noisy-alzheimer_acetyl_part1,177,177,354,/Users/yasmineakaichi/fed-popper/fedpopper/noi...
1,fedpopper,trains2,792,208,1000,/Users/yasmineakaichi/fed-popper/fedpopper/tra...
2,fedpopper,trains_part2,2,2,4,/Users/yasmineakaichi/fed-popper/fedpopper/tra...
3,fedpopper,carcinogenesis,162,136,298,/Users/yasmineakaichi/fed-popper/fedpopper/car...
4,fedpopper,trains_part3,1,1,2,/Users/yasmineakaichi/fed-popper/fedpopper/tra...
5,fedpopper,synthesis-contains,20,20,40,/Users/yasmineakaichi/fed-popper/fedpopper/syn...
6,fedpopper,trains2_part3,264,69,333,/Users/yasmineakaichi/fed-popper/fedpopper/tra...
7,fedpopper,iggp-rps_part3,36,118,154,/Users/yasmineakaichi/fed-popper/fedpopper/igg...
8,fedpopper,iggp-rps_part2,36,119,155,/Users/yasmineakaichi/fed-popper/fedpopper/igg...
9,fedpopper,trains2_part2,264,69,333,/Users/yasmineakaichi/fed-popper/fedpopper/tra...


In [5]:
df_summary

""


# Data Partitionning 

In [2]:
import os
import shutil
import re
import random

def partition_data(kbpath, num_parts=2, shuffle=True):
    """
    Partitionne exs.pl en fichiers balanced, avec POS avant NEG dans chaque partition.
    """
    ex_file = os.path.join(kbpath, "exs.pl")
    bk_file = os.path.join(kbpath, "bk.pl")
    bias_file = os.path.join(kbpath, "bias.pl")

    # --- Lire tous les exemples POS / NEG ---
    with open(ex_file, "r") as f:
        lines = f.readlines()

    pos = [l for l in lines if l.strip().startswith("pos")]
    neg = [l for l in lines if l.strip().startswith("neg")]

    # --- Optionnel : mélanger l’ordre des exemples ---
    if shuffle:
        random.shuffle(pos)
        random.shuffle(neg)

    # --- Split équilibré ---
    pos_splits = [pos[i::num_parts] for i in range(num_parts)]
    neg_splits = [neg[i::num_parts] for i in range(num_parts)]

    # --- Créer les dossiers partitions ---
    new_dirs = []
    for i in range(num_parts):
        part_dir = f"{kbpath}_part{i+1}"
        os.makedirs(part_dir, exist_ok=True)

        # Copy bk.pl & bias.pl
        shutil.copy(bk_file, os.path.join(part_dir, "bk.pl"))
        shutil.copy(bias_file, os.path.join(part_dir, "bias.pl"))

        # Write balanced exs.pl (POS first then NEG)
        out_file = os.path.join(part_dir, "exs.pl")
        with open(out_file, "w") as f:
            # always POS first, then NEG
            for l in pos_splits[i]:
                f.write(l)
            for l in neg_splits[i]:
                f.write(l)

        new_dirs.append(part_dir)

        print(f"📂 Created {part_dir}: {len(pos_splits[i])} POS, {len(neg_splits[i])} NEG")

    return new_dirs


In [3]:
partition_data("noisy-alzheimer_acetyl", num_parts=3)

📂 Created noisy-alzheimer_acetyl_part1: 177 POS, 177 NEG
📂 Created noisy-alzheimer_acetyl_part2: 177 POS, 177 NEG
📂 Created noisy-alzheimer_acetyl_part3: 176 POS, 176 NEG


['noisy-alzheimer_acetyl_part1',
 'noisy-alzheimer_acetyl_part2',
 'noisy-alzheimer_acetyl_part3']